In [1]:
import numpy as np
import cv2
static_back = None
video = cv2.VideoCapture('55_trim.mp4')
length = int (video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int (video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int (video.get(cv2.CAP_PROP_FRAME_HEIGHT))
kernel3 = np.ones((3,3),np.uint8)
out = cv2.VideoWriter('ss.mp4', cv2.VideoWriter_fourcc(*'MP4V'),24.0,(width,height))

k=0
fr_b = None

print(length)
while (video.isOpened() and k < length):
    k+=1
    check,frame = video.read()
    motion = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (7,7),0)

    if static_back is None:
        static_back = gray
        fr_b = gray
        continue
    else:
        static_back = gray
        
            
    
    diff_frame = cv2.absdiff(static_back, fr_b)
    fr_b = static_back
    
    thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations =10)
    crops = []
    kol_vo=0
    #fgmask = fgbg.apply(thresh_frame)
    cnts, _ = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in cnts:
        if cv2.contourArea(contour) > 1000000 or cv2.contourArea(contour) < 2500:
            continue
        
        motion=1
        kol_vo=kol_vo+1
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (155, 50, 165), 3)
        crop = frame[y:y+h, x:x+w]
        crops.append(crop)
    #cv2.imshow('Color frame', frame)
    #cv2.imshow('frame', frame)
    thresh_frame = cv2.cvtColor(thresh_frame, cv2.COLOR_GRAY2RGB)
    out.write(frame)
    
    #out.write(frame)

video.release()
out.release()
cv2.destroyAllWindows()

90


In [2]:
import torch
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
#ssd_model =torch.load('tensors.pt', map_location=torch.device('cpu'))

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [3]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [4]:
ssd_model.to('cuda')
ssd_model.eval()

SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [5]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input

In [6]:
import numpy as np
import cv2
static_back = None
video = cv2.VideoCapture('55_trim.mp4')
length = int (video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int (video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int (video.get(cv2.CAP_PROP_FRAME_HEIGHT))
kernel3 = np.ones((3,3),np.uint8)
out = cv2.VideoWriter('ss1.mp4', cv2.VideoWriter_fourcc(*'MP4V'),24.0,(width,height))

k=0
fr_b = None

ii = 0
while (video.isOpened() and k < length):
    k+=1
    check,frame = video.read()
    motion = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if static_back is None:
        static_back = gray
        fr_b = gray
        continue
    else:
        static_back = gray
    diff_frame = cv2.absdiff(static_back, fr_b)
    fr_b = static_back
    
    thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations =10)
    crops = []
    kol_vo=0
    cnts, _ = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if cv2.contourArea(contour) > 1000000 or cv2.contourArea(contour) < 2500:
            continue
        ii = ii + 1
        motion=1
        kol_vo=kol_vo+1
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (155, 50, 165), 3)
        crop = frame[y:y+h, x:x+w]
        path = "crop"+ str(ii) + ".jpg"
        cv2.imwrite(path, crop)
        inputs = [utils.prepare_input(path)]
        tensor = utils.prepare_tensor(inputs, precision == 'fp16')
        with torch.no_grad():
            detections_batch = ssd_model(tensor)
        if ii!=221:
          results_per_input = utils.decode_results(detections_batch)
          best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]
          classes_to_labels = utils.get_coco_object_dictionary()
          bboxes, classes, confidences = best_results_per_input[0]
          if len(classes) != 0:
            a = classes[0]
            if (a-1!=0):
                cv2.putText(frame, "not person", (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
            else:
              cv2.putText(frame, classes_to_labels[a-1], (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
          else:
            cv2.putText(frame, "not person", (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
    thresh_frame = cv2.cvtColor(thresh_frame, cv2.COLOR_GRAY2RGB)
    out.write(frame)

video.release()
out.release()
cv2.destroyAllWindows()

In [7]:
from tensorflow.keras.applications import ResNet152 
model2 = ResNet152()

In [8]:
import numpy as np
import cv2
static_back = None
video = cv2.VideoCapture('55_trim.mp4')
length = int (video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int (video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int (video.get(cv2.CAP_PROP_FRAME_HEIGHT))
kernel3 = np.ones((3,3),np.uint8)
out = cv2.VideoWriter('ss3.mp4', cv2.VideoWriter_fourcc(*'MP4V'),24.0,(width,height))
kol_pers = 0
k=0
fr_b = None
arr5 = []
ii = 0
while (video.isOpened() and k < length):
    k+=1
    check,frame = video.read()
    motion = 0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.GaussianBlur(gray, (7,7),0)

    if static_back is None:
        static_back = gray
        fr_b = gray
        continue
    else:
        static_back = gray
        
            
    
    diff_frame = cv2.absdiff(static_back, fr_b)
    fr_b = static_back
    
    thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)[1]
    thresh_frame = cv2.dilate(thresh_frame, None, iterations =10)
    crops = []
    kol_vo=0
    #fgmask = fgbg.apply(thresh_frame)
    cnts, _ = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in cnts:
        if cv2.contourArea(contour) > 1000000 or cv2.contourArea(contour) < 2500:
            continue
        ii = ii + 1
        motion=1
        kol_vo=kol_vo+1
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (155, 50, 165), 3)
        crop = frame[y:y+h, x:x+w]
        path = "crop"+ str(ii) + ".jpg"
        cv2.imwrite(path, crop)
        inputs = [utils.prepare_input(path)]
        tensor = utils.prepare_tensor(inputs, precision == 'fp16')
        with torch.no_grad():
            detections_batch = ssd_model(tensor)
        if ii!=221:
          results_per_input = utils.decode_results(detections_batch)
          best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]
          classes_to_labels = utils.get_coco_object_dictionary()
          bboxes, classes, confidences = best_results_per_input[0]
          if len(classes) != 0:
            a = classes[0]
            if (a-1!=0):
              if (a-1==16):
                image = load_img(path, target_size=(224, 224))
                image = img_to_array(image)
                image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
                image = preprocess_input(image)
                yhat = model2.predict(image)
                from tensorflow.keras.applications.resnet import decode_predictions
                label = decode_predictions(yhat)
                label = label[0][0]
                cv2.putText(frame, "not person", (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
                cv2.putText(frame, label[1], (x, y+h+30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
                arr5.append(["cadr " + str(k),x+w,y+h, "not person",label[1]])
              else:
                cv2.putText(frame, "not person", (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
                arr5.append(["cadr " + str(k),x+w,y+h, "not person"])
            else:
              cv2.putText(frame, classes_to_labels[a-1], (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
              kol_pers = kol_pers + 1
              arr5.append(["cadr " + str(k),x+w,y+h, "person"])
          else:
            cv2.putText(frame, "not person", (x, y+h), cv2.FONT_HERSHEY_PLAIN, 1, (50, 60, 10), 2)
            arr5.append(["cadr " + str(k),x+w,y+h, "not person"])
    #cv2.imshow('Color frame', frame)
    #cv2.imshow('frame', frame)
    thresh_frame = cv2.cvtColor(thresh_frame, cv2.COLOR_GRAY2RGB)
    out.write(frame)
    
    #out.write(frame)

video.release()
out.release()
cv2.destroyAllWindows()
print(arr5)

[['cadr 2', 250, 364, 'not person'], ['cadr 2', 106, 372, 'not person', 'Labrador_retriever'], ['cadr 2', 188, 302, 'person'], ['cadr 3', 251, 366, 'not person'], ['cadr 3', 102, 372, 'not person'], ['cadr 3', 188, 364, 'person'], ['cadr 4', 253, 367, 'not person'], ['cadr 4', 99, 371, 'not person'], ['cadr 4', 188, 358, 'not person'], ['cadr 5', 256, 366, 'not person'], ['cadr 5', 96, 373, 'not person', 'Labrador_retriever'], ['cadr 5', 189, 361, 'person'], ['cadr 6', 258, 367, 'not person'], ['cadr 6', 93, 373, 'not person', 'Labrador_retriever'], ['cadr 6', 189, 363, 'person'], ['cadr 7', 259, 370, 'not person'], ['cadr 7', 90, 372, 'not person', 'kuvasz'], ['cadr 7', 189, 363, 'person'], ['cadr 8', 261, 368, 'not person'], ['cadr 8', 87, 372, 'not person'], ['cadr 8', 188, 361, 'person'], ['cadr 9', 263, 368, 'not person'], ['cadr 9', 83, 373, 'not person'], ['cadr 9', 187, 366, 'person'], ['cadr 10', 265, 369, 'not person'], ['cadr 10', 79, 373, 'not person', 'Bedlington_terrier']

In [9]:
print(k)
print(kol_pers)
print((kol_pers/float(k))*100)

90
81
90.0


In [10]:
with open("log.txt", "w") as file:
    for  line in arr5:
      for  l in line:
        file.write(str(l) + " ")
      file.write('\n')